# DS-SF-34 | 10 | Regularization | Codealong | Starter Code

In [ ]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import preprocessing, linear_model, model_selection, metrics

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Part A | Closed-form solution for $\hat{\beta}$

In [ ]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-10-zillow.csv'), index_col = 'ID')

In [ ]:
df.dropna(subset = ['Size', 'SalePrice'], inplace = True)

train_X = df[ ['Size'] ]
train_X = sm.add_constant(train_X)

train_y = df.SalePrice

smf.OLS(train_y, train_X).fit().summary()

In [ ]:
# TODO

## Part B | Closed-form solution for $\hat{\beta}$ (cont.)

In [ ]:
train_x = train_X.Size

# TODO

print 'beta_0_hat =', beta_0_hat
print 'beta_1_hat =', beta_1_hat

## Part C | Regularization

### Truth

In [ ]:
def f(x):
    return np.sin(2 * np.pi * x)

In [ ]:
truth_df = pd.DataFrame(index = range(100))
truth_df['x'] = np.linspace(0, 1, truth_df.shape[0])
truth_df['y'] = truth_df.x.apply(f)

### Training/testing sets

- (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
np.random.seed(0)

df = pd.DataFrame(index = range(100))
df['x'] = np.random.uniform(0, 1, size = df.shape[0])

df['Noise'] = np.random.normal(size = df.shape[0])
df['y'] = df.x.apply(f) * (1 + .5 * df.Noise)

In [ ]:
X = df[ ['x'] ]
y = df.y

In [ ]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, train_size = .25)

In [ ]:
plt.figure()
plt.xlim((truth_df.x.min(), truth_df.x.max()))
plt.ylim((-2, 2))

plt.plot(truth_df.x, truth_df.y, color = 'green', label = 'truth')
plt.scatter(train_X.x, train_y, s = 10, label = 'train')

plt.legend()

### Polynomial Models

- (http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)
- (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)

In [ ]:
class Polynomial(object):
    def __init__(self, n):
        self.features = preprocessing.PolynomialFeatures(n)
        self.model = linear_model.LinearRegression()

    def fit(self, X, y):
        self.X = X
        # TODO

        return self

    def predict(self, X):
        # TODO

    def score(self, X, y):
        X = self.features.transform(X)
        return np.log(2 - self.model.score(X, y))

    def mean_squared_error(self, X, y):
        y_hat = self.predict(X)
        return np.log(metrics.mean_squared_error(y_hat, y))

In [ ]:
ns = [0, 1, 2, 3, 5, 7, 9, 11, 13, 17]

models = [Polynomial(n).fit(train_X, train_y) for n in ns]

### Plots

In [ ]:
plt.figure(figsize = (8, 8))
plt.xlim((truth_df.x.min(), truth_df.x.max()))
plt.ylim((-2, 2))

plt.plot(truth_df.x, truth_df.y, color = 'green', label = 'truth')
plt.scatter(train_X.x, train_y, s = 10, label = 'train')

for i, n in enumerate(ns):
    X = truth_df[ ['x'] ]
    y_hat = models[i].predict(X)
    plt.plot(X.x, y_hat, label = 'n = {}'.format(n))

plt.legend()

In [ ]:
score_df = pd.DataFrame({'n': ns, 'model': models})
score_df['train (log(2 - R^2))'] = score_df.model.apply(lambda model: model.score(train_X, train_y))
score_df['test (log(2 - R^2))'] = score_df.model.apply(lambda model: model.score(test_X, test_y))

score_df.drop('model', axis = 1).set_index('n').plot()

In [ ]:
mean_squared_error_df = pd.DataFrame({'n': ns, 'model': models})

mean_squared_error_df['train (log(MSE))'] = mean_squared_error_df.model.\
    apply(lambda model: model.mean_squared_error(train_X, train_y))

mean_squared_error_df['test (log(MSE))'] = mean_squared_error_df.model.\
    apply(lambda model: model.mean_squared_error(test_X, test_y))

mean_squared_error_df.drop('model', axis = 1).set_index('n').plot()

### Regularization

- (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
- (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

#### Ridge

In [ ]:
class RidgePolynomial(object):
    def __init__(self, alpha, n):
        self.features = preprocessing.PolynomialFeatures(n)
        self.model = linear_model.Ridge(alpha)

    def fit(self, X, y):
        self.X = X
        self.features.fit(X)

        X = self.features.transform(X)
        self.model.fit(X, y)

        return self

    def predict(self, X):
        X = self.features.transform(X)
        return self.model.predict(X)

    def score(self, X, y):
        X = self.features.transform(X)
        return self.model.score(X, y)

    def mean_squared_error(self, X, y):
        y_hat = self.predict(X)
        return metrics.mean_squared_error(y_hat, y)

    def complexity(self):
        return np.sqrt(np.sum((self.model.coef_) ** 2))

In [ ]:
alphas = [1e-1, 1e-2, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]

models = [RidgePolynomial(alpha, 17).fit(train_X, train_y) for alpha in alphas]

In [ ]:
plt.figure(figsize = (8, 8))
plt.xlim((truth_df.x.min(), truth_df.x.max()))
plt.ylim((-2, 2))

plt.plot(truth_df.x, truth_df.y, color = 'green', label = 'truth')
plt.scatter(train_X.x, train_y, s = 10, label = 'train')

for i, alpha in enumerate(alphas):
    X = truth_df[ ['x'] ]
    y_hat = models[i].predict(X)
    plt.plot(X.x, y_hat, label = 'alpha = {}'.format(alpha))

plt.legend()

In [ ]:
model_df = pd.DataFrame({'alpha': alphas, 'model': models})
model_df['R^2 (train)'] = model_df.model.apply(lambda model: model.score(train_X, train_y))
model_df['R^2 (test)'] = model_df.model.apply(lambda model: model.score(test_X, test_y))
model_df['Complexity'] = model_df.model.apply(lambda model: model.complexity())

model_df.drop('model', axis = 1).set_index('alpha')

In [ ]:
models[4].model.coef_

#### Lasso

In [ ]:
class LassoPolynomial(object):
    def __init__(self, alpha, n):
        self.features = preprocessing.PolynomialFeatures(n)
        self.model = linear_model.Lasso(alpha)

    def fit(self, X, y):
        self.X = X
        self.features.fit(X)

        X = self.features.transform(X)
        self.model.fit(X, y)

        return self

    def predict(self, X):
        X = self.features.transform(X)
        return self.model.predict(X)

    def score(self, X, y):
        X = self.features.transform(X)
        return self.model.score(X, y)

    def mean_squared_error(self, X, y):
        y_hat = self.predict(X)
        return metrics.mean_squared_error(y_hat, y)

    def complexity(self):
        return np.sum(np.abs(self.model.coef_))

In [ ]:
alphas = [1e-1, 1e-2, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]

models = [LassoPolynomial(alpha, 17).fit(train_X, train_y) for alpha in alphas]

In [ ]:
plt.figure(figsize = (8, 8))
plt.xlim((truth_df.x.min(), truth_df.x.max()))
plt.ylim((-2, 2))

plt.plot(truth_df.x, truth_df.y, color = 'green', label = 'truth')
plt.scatter(train_X.x, train_y, s = 10, label = 'train')

for i, alpha in enumerate(alphas):
    X = truth_df[ ['x'] ]
    y_hat = models[i].predict(X)
    plt.plot(X.x, y_hat, label = 'alpha = {}'.format(alpha))

plt.legend()

In [ ]:
model_df = pd.DataFrame({'alpha': alphas, 'model': models})
model_df['R^2 (train)'] = model_df.model.apply(lambda model: model.score(train_X, train_y))
model_df['R^2 (test)'] = model_df.model.apply(lambda model: model.score(test_X, test_y))
model_df['Complexity'] = model_df.model.apply(lambda model: model.complexity())

model_df.drop('model', axis = 1).set_index('alpha')

In [ ]:
models[-1].model.coef_